# Using pymysql to connect to server database
    Quering the database and giving output as in Result Table along with the Graphical Representation of the query result

    Importing all the required libraries

In [ ]:
import pymysql
import pandas as pd
import time
from matplotlib import pyplot as plot
import numpy as np

    Connecting to the MYSQL server using pymysql library

In [ ]:
server = pymysql.connect(host = "ec2-54-190-28-80.us-west-2.compute.amazonaws.com",user = "rem-con", passwd = "root",autocommit=True)
cursor = server.cursor()
print("connected")

    Creating database "house_list" to store all the values in the table "house_info"

In [ ]:
sql = "CREATE DATABASE IF NOT EXISTS house_list;"
cursor.execute(sql)
sql = "USE house_list;"
cursor.execute(sql)
sql = '''CREATE TABLE IF NOT EXISTS house_info(id integer not null,
 dataset varchar(50) not null,area integer,baths integer,beds integer,
 price integer not null, street varchar(150) not null,city varchar(50) not null,
 state varchar(20) not null,pincode integer,primary key(id));'''
cursor.execute(sql)
sql = 'show tables;'
cursor.execute(sql)
print(cursor.fetchall())

    Inserting all content in the table

In [ ]:

df = pd.read_csv("/Users/kaizer/Desktop/scraped_century21_data.csv")
df['price'] = df['price'].replace({'\$': '', ',': '','Ask':0}, regex=True)
df['area'] = df['area'].replace({'\$': '', ',': '','Ask':0}, regex=True)
df['street'] = df['street'].replace({'\$': '', "'": '','Ask':0}, regex=True)
df[['price']] = df[['price']].astype(int)
df[['baths']] = df[['baths']].astype(int)
df[['beds']] = df[['beds']].astype(int)
df[['pin']] = df[['pin']].astype(int)
#print(df.head())
time.sleep(5)
for i in range(len(df)):
    #print(i)
    sql = "INSERT INTO house_info VALUES('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');".format(i+1,df['dataset'][i],df['area'][i],df['baths'][i],df['beds'][i],df['price'][i],df['street'][i],df['city'][i],df['state'][i],df['pin'][i])
    cursor.execute(sql)
    print(sql)

    Quering the database to get relevant results of what user needs, giving result table as output along with the   graphical representation

In [ ]:
result =[]
result_table = []
server = pymysql.connect(host = "ec2-54-190-28-80.us-west-2.compute.amazonaws.com",user = "rem-con", passwd = "root",autocommit=True)
cursor = server.cursor()
sql = "USE house_list;"
cursor.execute(sql)
sql = '''select street,city from house_info where baths>0 and baths<2 ORDER BY city ASC;'''
cursor.execute(sql)
result = (cursor.fetchall())
for i in range(len(result)):
    x = {"street": result[i][0],"city":result[i][1]}
    result_table.append(x)
df = pd.DataFrame(result_table)
df1 = df.groupby(['city'],as_index=False).count()
plot.figure(figsize=(20,20))
width = 0.6
plot.barh(df1.city,df1.street,width, label ='Number of houses having Bath = 1')
for a,b in zip(df1.street,df1.city):
    plot.text(a,b, str(a))
plot.xlabel('total house in that area')
plot.ylabel('cities')
plot.legend()
plot.savefig('number-of-houses-having-bath-1.png')
plot.show()
